In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from imblearn.over_sampling import ADASYN
from skfeature.function.similarity_based import fisher_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.model_selection import StratifiedKFold
import graphviz
from sklearn.tree import export_graphviz

#import data from csv
ad = pd.read_csv('D:\Spring2017\Artificial Intelligence\Lymph.csv')
ad.head()


#get all the features
X_data = ad.iloc[:, 0:27]
X = pd.DataFrame(X_data)

#Print the colums name
print("Features before feature selection: {}".format(X.columns.values))

#Get classes
y_data = ad['Label']
y = pd.DataFrame(y_data)
y=y.values.ravel()

#Save the resmapling data into npy
X_resampled = np.load('lymph_x.npy')
y_resampled = np.load('lymph_y.npy')


cv=StratifiedKFold(n_splits=14)
from skfeature.function.similarity_based import reliefF

for train, test in cv.split(X_resampled, y_resampled):
    score = reliefF.reliefF(X_resampled[train], y_resampled[train])

print(score)
idx = reliefF.feature_ranking(score)

X_resampled = pd.DataFrame(X_resampled)
X_resampled.columns = X.columns.values

X1 = X_resampled.iloc[:, [idx[0], idx[1], idx[2], idx[3], idx[4], idx[5], idx[6], idx[7], idx[8], idx[9], idx[10], idx[11]]]

#X1 = X_resampled.iloc[:, [idx[0], idx[1], idx[2], idx[3], idx[4],idx[5]]]

#print(X_resampled.columns.values)

X_resampled = X1

clf = DecisionTreeClassifier(random_state=0)
cv=StratifiedKFold(n_splits=14)

scores = cross_val_score(clf, X_resampled, y_resampled, scoring="roc_auc", cv=cv)
acc = cross_val_score(clf, X_resampled, y_resampled, scoring="accuracy", cv=cv)
recall = cross_val_score(clf, X_resampled, y_resampled, scoring="recall", cv=cv)
prec = cross_val_score(clf, X_resampled, y_resampled, scoring="precision", cv=cv)

#acc = cross_val_score(classifier, X_resampled, y_resampled, scoring="accuracy", cv=cv)
#print(scores)
print("Accuracy {}".format(acc.mean()))
print("AUC {}".format(scores.mean()))
print("Recall {}".format(recall.mean()))
print("Precision {}".format(prec.mean()))

for train, test in cv.split(X_resampled, y_resampled):
    probas_ = clf.fit(X_resampled.iloc[train], y_resampled[train]).predict_proba(X_resampled.iloc[test])

export_graphviz(clf, out_file="Lymph_Relief.dot", class_names = [ "Non-Response", "Response"] ,
                feature_names=X_resampled.columns.values, impurity=False, filled=True
                )

with open("Lymph_Relief.dot") as f:
    dot_graph = f.read()

graphviz.Source(dot_graph)